In [1]:
import mlrun
project = mlrun.get_or_create_project(name='mlops', user_project=True, context='./')

> 2023-01-13 10:39:00,056 [info] loaded project mlops from MLRun DB


In [5]:
fn = mlrun.code_to_function(name='dalex', kind='job', handler='run_dalex',filename='src/dalex.py')

In [6]:
out = fn.run(local=True,
       params = {'df_train': project.get_artifact('get_data_housing').to_dataitem(),
                 'target': 'MEDV'})

> 2023-01-13 10:40:33,619 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-13 10:40:33,670 [info] starting run dalex-run_dalex uid=61403ab575f64f05a6e7976445c45d80 DB=http://mlrun-api:8080
dataframe shape before dalex : (404, 14)
Preparation of a new explainer is initiated

  -> data              : 404 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 404 values
  -> model_class       : xgboost.sklearn.XGBRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x7f3c8c86d550> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 4.99, mean = 21.8, max = 50.0
  -> model type        : regression will be used (default)
  -> residual

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...45c45d80,0,Jan 13 10:40:33,completed,dalex-run_dalex,kind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,,df_train=s3://mlrun/projects/mlops-jovyan/artifacts/5a3597024b7b403f80c7cde07a696f07/get_data/0/housing.csvtarget=MEDV,,dalex_outputtrain_datatest_data


> 2023-01-13 10:40:52,166 [info] run executed, status=completed


In [10]:
import json
params = {"model_class": "xgboost.XGBRegressor",
           "label_columns": "MEDV",
           "model_name": 'housing2',
           "random_state": 50}

for key,val in json.loads(project.get_artifact('dalex-run_dalex_dalex_output').to_dataitem().get()).items():
    params['sample_weight'] = val
    
train = mlrun.import_function('hub://auto_trainer')
train.run(handler='train',
       inputs={"dataset":  project.get_artifact('dalex-run_dalex_train_data').target_path,
               "test_set": project.get_artifact('dalex-run_dalex_test_data').target_path},
       params = params,
       local=True)

> 2023-01-13 10:41:58,429 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-13 10:41:58,450 [info] starting run auto-trainer-train uid=0013eef9342f4e9ab7709ec4feefb6c0 DB=http://mlrun-api:8080
> 2023-01-13 10:41:58,660 [info] Sample set not given, using the whole training set as the sample set
> 2023-01-13 10:41:58,773 [info] training 'housing2'


> 2023-01-13 10:42:00,201 [info] run executed, status=completed


## French motor dataset

In [ ]:
out = fn.run(local=True,
       params = {'df_train': project.get_artifact('get_data_motor').to_dataitem(),
                 'target': 'ClaimNb'})

> 2023-01-13 10:42:09,684 [info] starting run dalex-run_dalex uid=bd2d9261ac5f424cba243526f0d72dd2 DB=http://mlrun-api:8080
dataframe shape before dalex : (542409, 11)
Preparation of a new explainer is initiated

  -> data              : 542409 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 542409 values
  -> model_class       : xgboost.sklearn.XGBRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x7f3c8c86d550> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = -0.111, mean = 0.053, max = 2.02
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.844, mean = 5.24e-06, max = 15.8
  -> model_info        : pac

In [ ]:
params = {"model_class": "xgboost.XGBRegressor",
           "label_columns": "ClaimNb",
           "model_name": 'motor2',
           "random_state": 50}

for key,val in json.loads(project.get_artifact('dalex-run_dalex_dalex_output').to_dataitem().get()).items():
    params['sample_weight'] = val
    
train = mlrun.import_function('hub://auto_trainer')
train.run(handler='train',
       inputs={"dataset": project.get_artifact('dalex-run_dalex_train_data').target_path,
               "test_set": project.get_artifact('dalex-run_dalex_test_data').target_path},
       params = params,
       local=True)